# Header

In [1]:
#19-4-62
#https://grega.xyz/post/niapy_optimize_knn/  
%autosave 60
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import bisect
import math
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,confusion_matrix
import matplotlib.pyplot as plt

from IPython.core.debugger import set_trace
#set_trace()

import random
import gc



Autosaving every 60 seconds


# Controller to show result of Optimization

In [2]:
class StoreOptimization(object) :
    results = []
    generation = 0
    
    count_generation = 0
    
    store_best_weights = []
    store_best_error = 1
    
    #default
    n_generation_wanted = 1
    
    @staticmethod
    def static_N_Gen(val):
         StoreOptimization.n_generation_wanted = val
    
    @staticmethod
    def static_clearAll() :
        StoreOptimization.generation = 0
        StoreOptimization.count_generation = 0
        StoreOptimization.store_best_weights = []
        StoreOptimization.store_best_error = 1
        
    @staticmethod
    def static_appendString(head_str) :
        logging.info('%s' % (head_str))
        print('%s' % (head_str))
    
    @staticmethod
    def static_addResult(val,weights,n_fold) :
        
            
        StoreOptimization.count_generation = StoreOptimization.count_generation + 1
        StoreOptimization.generation = StoreOptimization.generation + 1
        
        if StoreOptimization.n_generation_wanted == StoreOptimization.count_generation :
            StoreOptimization.count_generation = 0
            StoreOptimization.static_appendString('fold %s nFES %s error %s' % (n_fold,StoreOptimization.generation, val))
        
        if val < StoreOptimization.store_best_error :
            StoreOptimization.store_best_error = val 
            StoreOptimization.store_best_weights = weights
            StoreOptimization.static_appendString('fold %s best at nFES %s error %s' % (n_fold,StoreOptimization.generation, val))
            logging.info(weights)

        
            


# Load Data

In [3]:
class leaf_node :
    def __init__(self,count_leaves) :
        self.list_weight_input = np.ones(count_leaves).tolist()
        self.gammar_operator = 1.0
        
    def cal_y(self,input_x) :
       
        
        if len(input_x) != len(self.list_weight_input) :
            print("Error cal , input not equal weight")
            return
        
        
        temp_mul_y1 = [ np.power(input_x[idx] , self.list_weight_input[idx]) for idx in range(0,len(input_x)) ]
        

        y1_out = 1
        for v in temp_mul_y1 : 
            y1_out = y1_out * v
            

        y1_out = np.power(y1_out,1-self.gammar_operator)
        
        
        temp_mul_y2 = [ np.power(1-input_x[idx],self.list_weight_input[idx]) for idx in range(0,len(input_x)) ]
        
        y2_out = 1
        for v in temp_mul_y2 : 
            y2_out = y2_out * v
            
        y2_out = np.power(1 - y2_out,self.gammar_operator)
        
        return y1_out * y2_out

    
    def change_weights(self,new_weight,new_gammar) :

        if self.count_weight() == len(new_weight) :
            self.list_weight_input = new_weight
            self.gammar_operator = new_gammar

        else :
            print("Error Update weight!!!!!")
            raise
            
    def count_weight(self) :
        return len(self.list_weight_input)
    
    def get_weights_gammar(self) :
        return self.list_weight_input + [self.gammar_operator]
        
            
        

In [4]:
class root_tree :
    
    def __init__(self,list_group) :
        self.number_layers = len(list_group)
        self.all_nodes = []
        

        #Leaf (Top) -> Root (Bottom)
        
        for count_layer in range(0,len(list_group)) :
            temp_leaves = []
            for count_node in  range(0,len(list_group[count_layer])) :
                temp_leaves.append(leaf_node(list_group[count_layer][count_node]))
            self.all_nodes.append(temp_leaves)
            
        #Split vector input to input each group
        self.group_input = list_group

    
    def get_all_weights_wGammar(self) :
        #Return weight and gammar all tree to vector
        output = []
    
        for each_layer in self.all_nodes :
            for each_node in each_layer :
                
                #New position vector X
                output = output+each_node.get_weights_gammar()
            
        return output
                
    def set_all_weights_wGammar(self,new_weights) :
        count_weight_idx = 0
        
        for each_layer in self.all_nodes :
            for each_node in each_layer :
                
                len_weight = each_node.count_weight()
                temp_weights = new_weights[count_weight_idx:count_weight_idx + len_weight]
                temp_gammar = new_weights[count_weight_idx+len_weight]
                
                
                #Change weights
                try : 
                    each_node.change_weights(temp_weights,temp_gammar)
                except :
                    print("Error change weight !")
                    sys.exit()
                
                #New position
                count_weight_idx = count_weight_idx+len_weight+1
        
        
                
          
    #For multi objective to find error
    def predict_eachLeaf(self,input_X) :
        
        summation_error = 0
        
        output_previous_layer = input_X
        
        summation_resultY = []
        
        #feed only leaves tree 
        for each_layer in self.all_nodes :

            count_idx = 0
            temp_previous_result = []
            
            for each_node in each_layer :
                
                input_x_vector = output_previous_layer[count_idx:count_idx + each_node.count_weight()]

                resultFrom_calY = each_node.cal_y(input_x_vector)

                summation_resultY =  summation_resultY + [resultFrom_calY]

                temp_previous_result.append(resultFrom_calY)

                #New position vector X
                count_idx = count_idx+each_node.count_weight()
            
            output_previous_layer =  temp_previous_result

        return summation_resultY
    
    
    #Feed input to get output only
    def predict(self,input_X) :
        return self.predict_eachLeaf(input_X)[-1]
        
    
    def readable(self) :
        #Return weight and gammar all tree to vector
        
        for (idx,each_layer) in enumerate(self.all_nodes) :
            for (i,each_node) in enumerate(each_layer) :
                print("Weight layer ",idx,":",i+1," ",each_node.get_weights_gammar()[0:-1])
                print("Gammar operator : ",each_node.get_weights_gammar()[-1])
                print("")
    
    def countAllNodesOutput(self) :
        output = 0
        for count_layer in self.group_input :
            output = output + len(count_layer)
        return output
                

    
                

        

# Train & Optimization

In [5]:



def try_div(x,y):
    try: 
        return x/y
    except ZeroDivisionError: 
        return 0
    
def swap_weight(val) :
    len_weight = len(val)
    temp_multiply =  try_div(len_weight,np.sum(val))
    if math.isnan(temp_multiply) or temp_multiply == 0 :
        temp_multiply = 1
        
    output = (temp_multiply * np.asarray(val)).tolist()
    output = [0 if math.isnan(x) else x for x in output]

    return output
    
def swap_gammar(val) :
    if math.isnan(val) :
        return 0
    return val

# our custom benchmark classs
class BenchmarkAllTree(object):
    def __init__(self,tree_shape,start_pos_f,end_pos_f,n_fold):
        # define lower bound of benchmark function
        self.Lower = 0
        # define upper bound of benchmark function
        self.Upper = 1
        
        self.tree_shape = tree_shape
        
        self.start_pos_f = start_pos_f
        self.end_pos_f = end_pos_f
        
        self.n_fold = n_fold
        
        self.fuzzy_system = fuzzy_aggregation(tree_shape = self.tree_shape,current_fold_train=n_fold)
        
        

    # function which returns evaluate function
    def function(self):
        def evaluate(D,sol):
            gc.collect()
            sol = np.abs(sol)

            all_weight = []

            last_idx = 0


            for each_layer in self.tree_shape :

                temp_previous_result = []

                for each_node in each_layer :
                    temp_weights = swap_weight(sol[last_idx:last_idx + each_node])
                    temp_gammar = swap_gammar(sol[last_idx + each_node])
                    last_idx = last_idx + each_node + 1
                    all_weight = all_weight + temp_weights + [temp_gammar]


            fitness = self.fuzzy_system.evaluate(all_weight,self.start_pos_f ,self.end_pos_f)
            StoreOptimization.static_addResult(fitness,all_weight,self.n_fold)

            if math.isnan(fitness) :
                set_trace()

            return fitness
        return evaluate
        



In [6]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

class fuzzy_aggregation :
    def __init__(self,tree_shape, seed=1235,current_fold_train=1,crossvalidation=4) :
        self.classifier_root_tree = root_tree(tree_shape)
        
        self.seed = seed
        self.current_fold_train = current_fold_train
        self.crossvalidation = crossvalidation
        
        global number_feature
        self.number_feature = number_feature
        self.all_shape = tree_shape 
        
        global all_data_train
        self.all_data_train = all_data_train

        random.seed(self.seed)
        idx = random.sample(range(0, len(self.all_data_train.index)), len(self.all_data_train.index))


        self.data_train,self.data_blindtest = self.split_blind_test(data_train=np.asarray(self.all_data_train)[idx,:])
        
        temp_X_train,temp_X_test = self.select_fold(self.current_fold_train)
        
        #Var for used
        self.X_train,self.y_train = self.split_features_class(temp_X_train)
        self.X_test,self.y_test = self.split_features_class(temp_X_test)
        self.X_blindtest,self.y_blindtest = self.split_features_class(self.data_blindtest)


        
    def split_blind_test(self,test_size=0.8,data_train = []) :
    
        range_split= math.floor(len(data_train)*test_size)
        return data_train[0:range_split,:], data_train[range_split:,:]
    
    def select_fold(self,number_fold):
        #return train,test
        range_v = math.floor(len(self.data_train)/self.crossvalidation)
        if number_fold == 1 :
            return self.data_train[range_v:,:] , self.data_train[0:range_v,:] 
        elif number_fold == self.crossvalidation :
            return self.data_train[0:range_v*(number_fold-1),:], self.data_train[range_v*(number_fold-1):,:] 
        else :
            temp_data_first = self.data_train[0:range_v*(number_fold-1),:]

            temp_data_second = self.data_train[range_v*(number_fold):,:]
            final_con = np.concatenate((temp_data_first, temp_data_second))
            return final_con , self.data_train[range_v*(number_fold-1):range_v*(number_fold),:]
    
    def split_features_class(self,v_input) :
        #Vector feature , Vector class
        return v_input[:,0:self.number_feature],v_input[:,-1]
    
    def calculate_RMSE(self,y_real,y_predict) :
        sum_error_each_data = 0
        for idx in range(0,len(y_real)) :
            dup_outputY = np.repeat(y_real[idx], len(y_predict[0]))
            temp = dup_outputY - y_predict[idx]
            result_error = np.dot(temp, temp)
            
            sum_error_each_data = sum_error_each_data + result_error
            

        #Root mean squared error (RMSE)
        return math.sqrt(sum_error_each_data/(len(y_real) * self.classifier_root_tree.countAllNodesOutput()))

    def predict(self,start_f_in,end_f_in) :
       
        return [ self.classifier_root_tree.predict_eachLeaf( self.X_train[idx,start_f_in:end_f_in+1] ) for idx in range(self.X_train.shape[0]) ]
        
    def evaluate(self,weight_gammar,start_f_in,end_f_in) :
        
        self.classifier_root_tree.set_all_weights_wGammar(weight_gammar)
        y_prediction = self.predict(start_f_in,end_f_in)

        #Root mean squared error (RMSE)
        return self.calculate_RMSE(self.y_train,y_prediction)
    
    def predict_validateTest(self,start_f_in,end_f_in) :
        return [ self.classifier_root_tree.predict_eachLeaf( self.X_test[idx,start_f_in:end_f_in+1]  ) for idx in range(self.X_test.shape[0]) ]
        
    
    def validate_test(self,weight_gammar,start_f_in,end_f_in) :
        self.classifier_root_tree.set_all_weights_wGammar(weight_gammar)
        
        y_prediction = self.predict_validateTest(start_f_in,end_f_in)
        
        return self.calculate_RMSE(self.y_test,y_prediction)
    
    def predict_blindtest(self,start_f_in,end_f_in) :
        return [ self.classifier_root_tree.predict_eachLeaf( self.X_blindtest[idx,start_f_in:end_f_in+1]  ) for idx in range(self.X_blindtest.shape[0]) ]
        
    
    def blind_test(self,weight_gammar,start_f_in,end_f_in) :
        self.classifier_root_tree.set_all_weights_wGammar(weight_gammar)
        
        y_prediction = self.predict_blindtest(start_f_in,end_f_in)
        
        return self.calculate_RMSE(self.y_blindtest,y_prediction)
    
    def print_results(self,weight_gammar,start_f_in,end_f_in) :
        self.classifier_root_tree.set_all_weights_wGammar(weight_gammar)
        
        y_prediction = [ self.classifier_root_tree.predict( self.X_blindtest[idx,start_f_in:end_f_in+1]  ) for idx in range(self.X_blindtest.shape[0]) ]
        
        error_all = []
        
        y_binary_prediction = []
        for (y1,y2) in zip(y_prediction,self.y_blindtest) :
            print("Predict %s , Actual %s " % (y1,y2))
            error_all.append(np.abs(y1-y2))
            if y1 >= 0.5 :
                y_binary_prediction.append(1)
            else :
                y_binary_prediction.append(0)
            
        print("MAE = ",np.sum(error_all)/len(error_all))

        tn, fp, fn, tp = confusion_matrix(self.y_blindtest,y_binary_prediction).ravel()
        
        print("tn, fp, fn, tp",tn, fp, fn, tp)
        acc = accuracy_score( self.y_blindtest,y_binary_prediction)
        f1 = f1_score(self.y_blindtest, y_binary_prediction, average='binary') 

        print("Acc = %s , f1 = %s" % (acc,f1))
        pow_error_all = [v*v for v in error_all]
        print("RMSE = ",math.sqrt(np.sum(pow_error_all)/len(pow_error_all)))
        
    def set_allWeights(self,new_weight) :
        self.classifier_root_tree.set_all_weights_wGammar(new_weight)
        
    def showAllWeights(self) :
        self.classifier_root_tree.readable()
    
            
        

# Setup log

In [7]:
# logger = logging.getLogger()
# fhandler = logging.FileHandler(filename='log_train_opt.log')
# formatter = logging.Formatter('%(asctime)s - %(message)s')
# fhandler.setFormatter(formatter)
# logger.addHandler(fhandler)
# logger.setLevel(logging.INFO)
import datetime 
class logging(object) :
    
    file_name = "log_train_Finan_1.log"
    
    def initial(file_name):
        logging.file_name = file_name
    
    @staticmethod
    def info(msg) :
        f = open(logging.file_name, "a+")
        f.write("%s - %s \n" % (datetime.datetime.now(),msg) )
        f.close()
        

# Train

In [8]:
from NiaPy.algorithms.basic import FlowerPollinationAlgorithm
from NiaPy.algorithms.basic import FireflyAlgorithm
from NiaPy.algorithms.basic import GreyWolfOptimizer

def count_weight_from_tree(shape_OfTree):
    count_weight = 0
    for each_layer in shape_OfTree :
        list_all_weight_gammar = [ w + 1 for w in each_layer ]
        count_weight = count_weight + np.sum(list_all_weight_gammar)

    return count_weight

tree_shape = [[8,8,8,8,8,8,8,8,8,8,8,8,8,8,8],[2,2,3,2,2,3,1],[1,2,2,1,1],[5]]
n_weight_gammar = count_weight_from_tree(tree_shape) 
logging.initial("log_train_Finan_NiaPy_FP_1.log")

# name_file_train = "./TestingModel/data_train_norm.csv"
name_file_train = "SMOTH_data_2train8_reorder.csv"
StoreOptimization.static_appendString("You're going to train with file " + name_file_train + " & Saved = "+logging.file_name)
all_data_train = pd.read_csv(name_file_train,index_col="index_firm")

##################### Store Best fold #########################

best_sol_cross = []
error_cross = 1
best_fold = 1


############## Start & End pos feature #######################
pos_start = 0
pos_end = 119


################# In Aggregation algor ##################
number_feature = 120
crossvalidation = 4

############### Config Params ###############
train_params = {}
train_params["NP"] = 50
train_params["nFES"] = 10000
train_params["dimension"] = n_weight_gammar


######Flower Pollination##########
train_params["p"] = 0.35


########## Firefly algor ##############
train_params["alpha"] = 0.1
train_params["betamin"] = 0.5
train_params["gamma"] = 2



# 'D' = D {array} or {int} – Shape of return random numbers 

for n_fold in range(1,crossvalidation+1) :
    current_fold_train = n_fold
    #nGEN , nFES
    StoreOptimization.static_clearAll()
    StoreOptimization.static_N_Gen(1000)
    
    
    #flower pollination
    StoreOptimization.static_appendString("## New training FP with D=%s,NP = %s, nFES=%s, p=%s, with fold = %s " % (train_params["dimension"],train_params["NP"], train_params["nFES"], train_params["p"],n_fold))
    algorithm = FlowerPollinationAlgorithm(D=train_params["dimension"],NP = train_params["NP"], nFES=train_params["nFES"], p=train_params["p"], benchmark=BenchmarkAllTree(tree_shape,pos_start,pos_end,current_fold_train))
    
    #firefly
#     StoreOptimization.static_appendString("## New training FF with D=%s,NP = %s, nFES=%s, alpha=%s, betamin=%s, gamma = %s with fold = %s " % (train_params["dimension"],train_params["NP"], train_params["nFES"], train_params["alpha"],train_params["betamin"],train_params["gamma"] ,n_fold))
#     algorithm = FireflyAlgorithm(D=train_params["dimension"] ,NP = train_params["NP"], nFES=train_params["nFES"], alpha=train_params["alpha"], betamin=train_params["betamin"], gamma=train_params["gamma"], benchmark=BenchmarkAllTree(tree_shape,pos_start,pos_end,current_fold_train))

    
    #GWO
#     StoreOptimization.static_appendString("## New training GWO with D=%s,NP = %s, nFES=%s, with fold = %s " % (train_params["dimension"],train_params["NP"], train_params["nFES"] ,n_fold))
#     algorithm = GreyWolfOptimizer(D=train_params["dimension"], NP=train_params["NP"], nFES=train_params["nFES"], benchmark=BenchmarkAllTree(tree_shape,pos_start,pos_end,current_fold_train))

    best = algorithm.run()
    
    print("Optimal Weight and Gammar are : ")  
    if best[1] != StoreOptimization.store_best_error :
        print("Best not equal we find")
        raise
    best_solution = StoreOptimization.store_best_weights
    best = StoreOptimization.store_best_error
    
    

    
    print(":: Finished Fold ::")

    error_output = fuzzy_aggregation(tree_shape=tree_shape,current_fold_train=n_fold).validate_test(best_solution,pos_start,pos_end)

    print("!! Train error = ",best," & Validate Test Acc = ",error_output)
    StoreOptimization.static_appendString("!! Train error = "+str(best)+" Current fold = " + str(current_fold_train) + " Validate Test Acc =  " + str(error_output))
    if error_output < error_cross :
        best_sol_cross = best_solution
        error_cross = error_output
        best_fold = n_fold
        
    StoreOptimization.static_appendString("!! Train error = "+str(best)+" Current fold = " + str(current_fold_train) + " Validate Test Acc =  " + str(error_output))
    StoreOptimization.static_appendString("** Best Weights now = " + str(best_sol_cross))
    StoreOptimization.static_appendString("** Best n_Fold now = " + str(best_fold))
    StoreOptimization.static_appendString("** Best error now = " + str(error_cross))
        
        
error_blind = fuzzy_aggregation(tree_shape).blind_test(best_sol_cross,pos_start,pos_end)
StoreOptimization.static_appendString(">> Best Weights = " + str(best_sol_cross))
StoreOptimization.static_appendString(">> Best n_Fold = " + str(best_fold))
StoreOptimization.static_appendString(">> Best validate test error = " + str(error_cross))
StoreOptimization.static_appendString(">> Blind Test error = " + str(error_blind))



You're going to train with file SMOTH_data_2train8_reorder.csv & Saved = log_train_Finan_NiaPy_FP_1.log
## New training FP with D=175,NP = 50, nFES=10000, p=0.35, with fold = 1 
fold 1 best at nFES 1 error 0.697073150613766
fold 1 best at nFES 6 error 0.6862373148404923


KeyboardInterrupt: 

In [ ]:
system_f = fuzzy_aggregation(tree_shape).print_results(best_sol_cross,pos_start,pos_end)


# Show Structure

In [ ]:
system_f = fuzzy_aggregation(tree_shape)
system_f.set_allWeights(best_sol_cross)
system_f.showAllWeights()

